---
#**1.  Data Cleaning / Data Preprocessing**

---


---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import skew, kurtosis
import warnings
import os
warnings.filterwarnings('ignore')

In [ ]:
# Configuration des graphiques
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_style("whitegrid")

sns.set_palette("husl")

In [ ]:
import pandas as pd

# Chemin relatif depuis le dossier racine du projet
df = pd.read_csv("/content/noisy_paddydataset.csv")


print("="*60)
print("APERÇU GÉNÉRAL DU DATASET")
print("="*60)
print(f"Dimensions du dataset: {df.shape}")
print(f"Nombre total de cellules: {df.shape[0] * df.shape[1]:,}")
print(df.head())
print("="*60)
print("APERÇU GÉNÉRAL DU DATASET")
print("="*60)
print(f"Dimensions du dataset: {df.shape}")
print(f"Nombre total de cellules: {df.shape[0] * df.shape[1]:,}")

In [ ]:
# Informations générales
print("INFORMATIONS GÉNÉRALES:")
print("-" * 30)
df.info()
print()

In [ ]:
# Statistiques descriptives de base
print("STATISTIQUES DESCRIPTIVES - VARIABLES NUMÉRIQUES:")
print("-" * 50)
print(df.describe())

In [ ]:
# ----------------------------------------------------------------
# 3. DISTINCTION VARIABLES NUMÉRIQUES ET CATÉGORIQUES
# ----------------------------------------------------------------

print("\n" + "-"*70)
print("DISTINCTION VARIABLES NUMÉRIQUES ET CATÉGORIQUES")
print("-"*70)

# Identification des types de variables
numeric_features = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = df.select_dtypes(include=['object']).columns.tolist()

print(f"Variables numériques: {len(numeric_features)}")
print(f"  Exemples: {numeric_features[:10]}")
print(f"\nVariables catégoriques: {len(categorical_features)}")
print(f"  Exemples: {categorical_features[:10]}")

In [ ]:
# ================================================================
# 2. ANALYSE DES DONNÉES MANQUANTES
# ================================================================

print("\n" + "="*60)
print("ANALYSE DES DONNÉES MANQUANTES")
print("="*60)

In [ ]:
# Calcul des valeurs manquantes
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100
missing_df = pd.DataFrame({
    'Colonnes': missing_data.index,
    'Valeurs_Manquantes': missing_data.values,
    'Pourcentage': missing_percent.values
}).sort_values('Pourcentage', ascending=False)

In [ ]:
# Affichage des colonnes avec des valeurs manquantes
missing_cols = missing_df[missing_df['Valeurs_Manquantes'] > 0]
print(f"Nombre de colonnes avec des valeurs manquantes: {len(missing_cols)}")
print("\nColonnes avec le plus de valeurs manquantes:")
print(missing_cols.head(10))

In [ ]:
# Visualisation des données manquantes
plt.figure(figsize=(15, 8))
missing_cols_top = missing_cols.head(15)
plt.barh(missing_cols_top['Colonnes'], missing_cols_top['Pourcentage'])
plt.xlabel('Pourcentage de valeurs manquantes')
plt.title('Top 15 des colonnes avec des valeurs manquantes')
plt.gca().invert_yaxis()


In [ ]:
# Heatmap des valeurs manquantes (échantillon)
sample_cols = missing_cols.head(10)['Colonnes'].tolist()
if sample_cols:
    sns.heatmap(df[sample_cols].isnull(), cbar=True, yticklabels=False, cmap='viridis')
    plt.title('Heatmap des valeurs manquantes\n(Top 10 colonnes)')
plt.tight_layout()
plt.show()

2.1 Visualisation : Histogrammes avec Mean & Median

In [ ]:
# Créer une grille d'histogrammes (max 3 par ligne)
n_cols = 3
n_rows = (len(numeric_features) + n_cols - 1) // n_cols

plt.figure(figsize=(20, 5 * n_rows))

for i, col in enumerate(numeric_features, 1):
    plt.subplot(n_rows, n_cols, i)
    data = df[col].dropna()

    sns.histplot(data, kde=True, stat="density", alpha=0.7, color='skyblue', linewidth=0)

    mean_val = data.mean()
    median_val = data.median()

    plt.axvline(mean_val, color='red', linestyle='--', linewidth=2, label=f'Moyenne: {mean_val:.2f}')
    plt.axvline(median_val, color='green', linestyle='-', linewidth=2, label=f'Médiane: {median_val:.2f}')

    skewness = stats.skew(data)
    plt.title(f'{col}\\nSkewness: {skewness:.2f}', fontsize=12)
    plt.xlabel(col)
    plt.ylabel('Densité')
    plt.legend(fontsize=9)

plt.tight_layout()
plt.suptitle('Distributions des Variables Numériques (avec Moyenne & Médiane)', fontsize=16, y=1.02)
plt.show()

2.2 Tests Statistiques & Suggestion d'Imputation

In [ ]:
# Analyse statistique pour chaque colonne numérique
imputation_suggestions = []

for col in numeric_features:
    data = df[col].dropna()
    n = len(data)

    if n < 3:
        continue  # Pas assez de données pour les tests

    mean = data.mean()
    median = data.median()
    skewness = stats.skew(data)
    kurt = stats.kurtosis(data)

    # Test de normalité selon taille d'échantillon
    if n <= 500:
        # Shapiro-Wilk (p-value > 0.05 → normal)
        shapiro_stat, shapiro_p = stats.shapiro(data)
        test_name = 'Shapiro-Wilk'
        p_value = shapiro_p
    elif n <= 5000:
        # Kolmogorov-Smirnov (données standardisées)
        standardized_data = (data - mean) / data.std()
        ks_stat, ks_p = stats.kstest(standardized_data, 'norm')
        test_name = 'Kolmogorov-Smirnov'
        p_value = ks_p
    else:
        # Trop grand dataset → on se fie à skewness
        p_value = np.nan
        test_name = 'N/A'

    # Décision d'imputation
    if abs(skewness) <= 0.5 and (np.isnan(p_value) or p_value > 0.05):
        imputation = "Moyenne (distribution symétrique et normale)"
    else:
        imputation = "Médiane (distribution asymétrique ou non normale)"

    imputation_suggestions.append({
        'Colonne': col,
        'Moyenne': round(mean, 2),
        'Médiane': round(median, 2),
        'Skewness': round(skewness, 3),
        'Test Normalité': test_name,
        'p-value': round(p_value, 4) if not np.isnan(p_value) else 'N/A',
        'Suggestion Imputation': imputation
    })

# Affichage des résultats
results_df = pd.DataFrame(imputation_suggestions)
results_df = results_df.sort_values(by='Skewness', key=abs, ascending=False)

print("RÉSUMÉ STATISTIQUE ET SUGGESTIONS D'IMPUTATION")
print("="*80)
print(results_df.to_string(index=False))

2.3 Application de l'Imputation Recommandée

In [ ]:
df_cleaned = df.copy()
# Imputation automatique selon les suggestions
for col in numeric_features:
    data = df_cleaned[col]
    if data.isnull().sum() == 0:
        continue  # Rien à imputer

    # Récupérer la suggestion
    suggestion = results_df.loc[results_df['Colonne'] == col, 'Suggestion Imputation'].values[0]

    if "Moyenne" in suggestion:
        df_cleaned[col].fillna(df_cleaned[col].mean(), inplace=True)
        print(f"{col} : imputé par la MOYENNE")
    else:
        df_cleaned[col].fillna(df_cleaned[col].median(), inplace=True)
        print(f"{col} : imputé par la MÉDIANE")

# Vérification finale
print(f"\nValeurs manquantes restantes (numériques) : {df_cleaned[numeric_features].isnull().sum().sum()}")

2.4 Détection des Outliers

In [ ]:
# Après imputation des valeurs aberrantes
from scipy.stats.mstats import winsorize

def auto_treat_outliers(df, numeric_features, min_rows_threshold=5000, plot=True):
    """
    Traite automatiquement les outliers en choisissant la méthode la plus adéquate.

    Critères de décision :
    - Si très peu d'outliers (<1%) et dataset grand → Suppression possible
    - Si skewness > 1 et valeurs >=0 → Transformation log
    - Si % outliers > 5% → Capping percentiles 5%/95%
    - Sinon → Robust IQR + remplacement par médiane
    """

    summary = []
    rows_to_drop = set()
    initial_rows = len(df)

    # Sauvegarde des données AVANT traitement (pour visualisation)
    df_before = df.copy()

    print("TRAITEMENT AUTOMATIQUE DES OUTLIERS - CHOIX INTELLIGENT PAR COLONNE")
    print("=" * 100)

    # ============================
    # TRAITEMENT COLONNE PAR COLONNE
    # ============================
    for col in numeric_features:
        data = df[col].dropna()

        if len(data) < 10:
            summary.append({
                'Colonne': col,
                'Méthode Choisie': 'Ignoré (trop peu de données)'
            })
            continue

        # Statistiques
        skewness = stats.skew(data)
        median = data.median()

        # Détection IQR
        Q1 = data.quantile(0.25)
        Q3 = data.quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR

        outlier_mask = (df[col] < lower) | (df[col] > upper)
        nb_outliers = outlier_mask.sum()
        percent_outliers = nb_outliers / len(df) * 100

        method_chosen = ""
        action = ""

        # ============================
        # DÉCISION AUTOMATIQUE
        # ============================
        if nb_outliers == 0:
            method_chosen = "Aucun outlier"
            action = "Rien à faire"

        elif percent_outliers < 1 and len(df) > min_rows_threshold:
            rows_to_drop.update(df.loc[outlier_mask].index)
            method_chosen = "Suppression"
            action = f"{nb_outliers} lignes supprimées"

        elif abs(skewness) > 1 and data.min() >= 0:
            df[col + "_log"] = np.log1p(df[col])
            method_chosen = "Transformation log"
            action = "Nouvelle colonne _log créée"

        elif percent_outliers > 5:
            df[col] = winsorize(df[col], limits=[0.05, 0.05])
            method_chosen = "Capping (5%/95%)"
            action = "Winsorization appliquée"

        else:
            df.loc[outlier_mask, col] = median
            method_chosen = "Robust IQR + médiane"
            action = f"{nb_outliers} outliers remplacés par médiane"

        summary.append({
            'Colonne': col,
            'Skewness': round(skewness, 3),
            '% Outliers': round(percent_outliers, 2),
            'Méthode Choisie': method_chosen,
            'Action': action
        })

    # ============================
    # SUPPRESSION FINALE (UNE FOIS)
    # ============================
    if rows_to_drop:
        df.drop(index=list(rows_to_drop), inplace=True)
        df.reset_index(drop=True, inplace=True)
        df_before = df_before.loc[df.index].reset_index(drop=True)

    # ============================
    # VISUALISATION AVANT / APRÈS
    # ============================
    if plot:
        fig, axes = plt.subplots(
            nrows=len(numeric_features),
            ncols=2,
            figsize=(14, 4 * len(numeric_features))
        )

        if len(numeric_features) == 1:
            axes = np.array([axes])

        for i, col in enumerate(numeric_features):
            sns.boxplot(
                y=df_before[col],
                ax=axes[i, 0],
                color='lightcoral'
            )
            axes[i, 0].set_title(f'Avant - {col}')

            sns.boxplot(
                y=df[col],
                ax=axes[i, 1],
                color='lightgreen'
            )
            axes[i, 1].set_title(f'Après - {col}')

        plt.suptitle("Comparaison AVANT / APRÈS par colonne", fontsize=16)
        plt.tight_layout()
        plt.show()

    # ============================
    # RÉCAPITULATIF
    # ============================
    final_rows = len(df)
    print(f"\nNombre de lignes : {initial_rows} → {final_rows}")

    summary_df = pd.DataFrame(summary)
    print("\nRÉCAPITULATIF DU TRAITEMENT AUTOMATIQUE")
    print(summary_df.to_string(index=False))

    return df, summary_df

In [ ]:
df_cleaned, recap = auto_treat_outliers(df_cleaned, numeric_features, plot=True)


print("\n=== DATAFRAME APRÈS TRAITEMENT ===")
display(df_cleaned)

In [ ]:
print("\n=== RÉCAPITULATIF ===")
display(recap)

In [ ]:
# ===================================================================
# 2.5 Imputation + Uniformisation des Variables Catégorielles
# ===================================================================

print("ÉTAT AVANT TRAITEMENT")
print("="*80)
for col in categorical_features:
    if col in df_cleaned.columns:
        missing = df_cleaned[col].isnull().sum()
        uniques = df_cleaned[col].nunique()
        print(f"{col:30} → {missing:3} valeurs manquantes | {uniques} valeurs uniques")

print("\n" + "="*80)
print("TRAITEMENT : IMPUTATION (mode) + UNIFORMISATION (minuscules + strip)")
print("="*80)

treatment_summary = []

for col in categorical_features:
    if col not in df_cleaned.columns:
        print(f"{col} : colonne non présente")
        continue

    initial_missing = df_cleaned[col].isnull().sum()
    initial_unique = df_cleaned[col].nunique()

    # 1. Imputation des valeurs manquantes par le mode
    if initial_missing > 0:
        mode_value = df_cleaned[col].mode(dropna=True)[0]
        df_cleaned[col].fillna(mode_value, inplace=True)
        imputed = True
    else:
        mode_value = None
        imputed = False

    # 2. Uniformisation : minuscules + suppression espaces
    df_cleaned[col] = df_cleaned[col].astype(str).str.lower().str.strip()

    # Après traitement
    final_missing = df_cleaned[col].isnull().sum()
    final_unique = df_cleaned[col].nunique()

    print(f"✓ {col}")
    if imputed:
        print(f"   → {initial_missing} valeurs manquantes imputées par mode : '{mode_value}'")
    print(f"   → Valeurs uniques : {initial_unique} → {final_unique} (réduction des doublons de casse)")

    treatment_summary.append({
        'Colonne': col,
        'Manquantes Initiales': initial_missing,
        'Imputé par Mode': mode_value if imputed else 'Non',
        'Uniques Avant': initial_unique,
        'Uniques Après': final_unique
    })

# Vérification finale
print("\n" + "="*80)
print("VÉRIFICATION FINALE")
print("="*80)
total_missing_cat = df_cleaned[categorical_features].isnull().sum().sum()
print(f"Valeurs manquantes restantes (catégorielles) : {total_missing_cat} → doit être 0")

print("\nValeurs uniques finales par colonne :")
for col in categorical_features:
    if col in df_cleaned.columns:
        print(f"   {col:30} → {df_cleaned[col].nunique()} valeurs : {sorted(df_cleaned[col].unique())[:10]}...")

# Tableau récapitulatif
summary_df = pd.DataFrame(treatment_summary)
print("\nRÉCAPITULATIF DU TRAITEMENT")
print(summary_df.to_string(index=False))

In [ ]:
# Créer le dossier data s'il n'existe pas
os.makedirs("data", exist_ok=True)
# Enregistrement en CSV
df_cleaned.to_csv("data/cleaned_paddydataset.csv", index=False)

print("\nDataFrame sauvegardé dans : data/cleaned_paddydataset.csv")